In [7]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"C:\Users\hamupratik\Downloads\937b47ca-125c-44c5-bfdd-0975c0647a6f.csv", dtype =str)


4219

In [8]:
df['plan_paid_amount'] = df['plan_paid_amount'].astype(float)
df['claim_received_date'] = pd.to_datetime(df['claim_received_date'], errors='coerce')
modifier_cols = ['proc_modifier', 'proc_modifier2', 'proc_modifier4', 'proc_modifier5']
df['procedure_code'] = df['procedure_code'].str.strip()
df[modifier_cols] = df[modifier_cols].fillna('').apply(lambda x: x.str.strip().str.lower())
df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')


In [9]:
import pandas as pd
import numpy as np

df['procedure_code'] = df['procedure_code'].str.strip()

# Group by member and procedure code
group_keys = ['member_medicare_id',
    'procedure_code',
    'proc_modifier',
    'proc_modifier2',
    'proc_modifier4',
    'proc_modifier5',
    'quantity']

def flag_by_procedure(group):
    group = group[group['plan_paid_amount'] > 75].sort_values('claim_received_date')

    if group.empty or len(group) == 1:
        return pd.Series(['other'] * len(group), index=group.index)

    first_date = group.iloc[0]['claim_received_date']
    last_date = group.iloc[-1]['claim_received_date']

    flags = []
    for _, row in group.iterrows():
        if first_date == last_date:
            flags.append('duplicate')
        elif row['claim_received_date'] == first_date:
            flags.append('reference')
        elif row['claim_received_date'] == last_date:
            flags.append('target')
        else:
            flags.append('other')
    return pd.Series(flags, index=group.index)

# Apply group-wise flagging
flags_series = df.groupby(group_keys, group_keys=False).apply(flag_by_procedure)

# Initialize and assign flags
df['proc_code_flag'] = 'other'
df.loc[flags_series.index, 'proc_code_flag'] = flags_series

# Optionally, set target_code where appropriate
df['target_code'] = df.apply(
    lambda row: row['procedure_code'] if row['proc_code_flag'] in ['target', 'duplicate'] else np.nan,
    axis=1
)


C:\Users\hamupratik\AppData\Local\Temp\ipykernel_14580\3919107775.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  flags_series = df.groupby(group_keys, group_keys=False).apply(flag_by_procedure)


In [10]:
df['proc_code_flag'].unique()

array(['target', 'reference', 'duplicate', 'other'], dtype=object)

In [11]:
# return 
# df